In [1]:
import os
import pandas as pd

dir = os.getcwd()

coffe_cities = pd.read_excel(os.path.join(dir, 'Data', 'municipios_seleccionados.xlsx'))
stations = pd.read_csv(os.path.join(dir, 'Data', 'Catálogo_Nacional_de_Estaciones_del_IDEAM_20251024.csv'))

FileNotFoundError: [Errno 2] No such file or directory: 'e:\\DATA ANALIST\\Git\\Proyecto Despliegue\\MIAD---PROYECTO---DESPLIEGUE-DE-SOLUCIONES\\Data\\Catálogo_Nacional_de_Estaciones_del_IDEAM_20251024.csv'

In [ ]:
# Unir por Municipio
stations_selected = pd.merge(
    coffe_cities[['municipio']],                # municipios seleccionados
    stations,
    left_on='municipio',
    right_on='Municipio',
    how='inner'                                 # coincidencias exactas
)
stations_selected = stations_selected[['Departamento','Municipio','Codigo','Nombre','Categoria','Tecnologia','LONGITUD','LATITUD','Altitud','Fecha_instalacion','Fecha_suspension']]

In [3]:
stations_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Departamento       172 non-null    object 
 1   Municipio          172 non-null    object 
 2   Codigo             172 non-null    int64  
 3   Nombre             172 non-null    object 
 4   Categoria          172 non-null    object 
 5   Tecnologia         172 non-null    object 
 6   LONGITUD           172 non-null    float64
 7   LATITUD            172 non-null    float64
 8   Altitud            172 non-null    object 
 9   Fecha_instalacion  171 non-null    object 
 10  Fecha_suspension   55 non-null     object 
dtypes: float64(2), int64(1), object(8)
memory usage: 14.9+ KB


In [ ]:
# 1) Parsear las fechas (maneja strings variados; 'errors="coerce"' pone NaT si no se puede parsear)
for col in ['Fecha_instalacion', 'Fecha_suspension']:
    if col in stations_selected.columns:
        stations_selected[col] = pd.to_datetime(
            stations_selected[col],
            errors='coerce',
            dayfirst=True
        )

# 2) Definir fecha de corte
corte = pd.Timestamp('2010-01-01')

# 3) Construir la máscara:
#    - Mantener registros sin fecha de suspensión (NaT)  -> estación activa o sin dato
#    - Mantener registros con suspensión >= 2010-01-01
mask = stations_selected['Fecha_suspension'].isna() | (stations_selected['Fecha_suspension'] >= corte)

# 4) Aplicar filtro
antes = len(stations_selected)
stations_selected = stations_selected.loc[mask].reset_index(drop=True)
despues = len(stations_selected)

print(f"Estaciones eliminadas por suspensión antes de 2010-01-01: {antes - despues}")
stations_selected.info()


In [2]:
import os
from pathlib import Path
import unicodedata
import pandas as pd

# --- utilidades ---
def strip_accents(s: str) -> str:
    if s is None: 
        return ""
    return "".join(ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn")

def find_station_file(data_dir: Path) -> Path | None:
    """
    Busca un archivo que parezca el catálogo de estaciones del IDEAM, 
    tolerando acentos y variaciones. Prefiere .csv si hay varias coincidencias.
    """
    target_tokens = ["catalogo", "catálogo", "nacional", "estaciones", "ideam", "20251024"]
    # Normaliza tokens sin acentos para comparar
    target_tokens = [strip_accents(t).casefold() for t in target_tokens]

    candidates = list(data_dir.glob("*"))
    scored = []
    for p in candidates:
        if not p.is_file():
            continue
        name_norm = strip_accents(p.name).casefold()
        score = sum(tok in name_norm for tok in target_tokens)
        if score >= 3:  # umbral razonable
            scored.append((score, p))

    if not scored:
        return None

    # Ordena por score (desc), prefiriendo .csv
    scored.sort(key=lambda t: (t[0], t[1].suffix.lower() == ".csv"), reverse=True)

    # Si hay varios, prioriza el .csv con mayor score
    best = None
    for _, p in scored:
        if p.suffix.lower() == ".csv":
            best = p
            break
    if not best:
        best = scored[0][1]
    return best

# --- rutas base ---
cwd = Path(os.getcwd())
data_dir = cwd / "Data"

# Carga municipios
coffe_cities_path = data_dir / "municipios_seleccionados.xlsx"
coffe_cities = pd.read_excel(coffe_cities_path)

# Localiza el archivo de estaciones de forma robusta
stations_path = find_station_file(data_dir)
if stations_path is None:
    # Si no se encontró, imprime listado para inspección
    print("No encontré el catálogo. Archivos en Data/:")
    for p in sorted(data_dir.glob("*")):
        print(" -", p.name)
    raise FileNotFoundError("No se encontró el archivo del catálogo IDEAM (revisa nombre/extensión).")

print(f"Archivo detectado: {stations_path.name}")

# Carga robusta (CSV o Excel)
if stations_path.suffix.lower() == ".csv":
    # Intento 1: inferencia de separador con engine=python
    tried = []
    try:
        stations = pd.read_csv(stations_path, sep=None, engine="python")
    except Exception as e1:
        tried.append(f"sep=None utf-8: {e1}")
        # Intento 2: codificaciones comunes
        loaded = False
        for enc in ["utf-8-sig", "latin1", "cp1252"]:
            for sep in [",", ";", "\t", "|"]:
                try:
                    stations = pd.read_csv(stations_path, encoding=enc, sep=sep)
                    loaded = True
                    print(f"Cargado con encoding={enc} sep='{sep}'")
                    break
                except Exception as e2:
                    tried.append(f"encoding={enc} sep='{sep}': {e2}")
            if loaded:
                break
        if not loaded:
            print("Intentos fallidos:\n - " + "\n - ".join(tried))
            raise
else:
    # Excel (por si el “CSV” realmente era xlsx/xls)
    stations = pd.read_excel(stations_path)

# --- Unir por municipio ---
stations_selected = pd.merge(
    coffe_cities[['municipio']],
    stations,
    left_on='municipio',
    right_on='Municipio',
    how='inner'
)

# --- Selección de columnas si existen (ignora las que no existan) ---
cols_pref = ['Departamento','Municipio','Codigo','Nombre','Categoria','Tecnologia',
             'LONGITUD','LATITUD','Altitud','Fecha_instalacion','Fecha_suspension']
cols_exist = [c for c in cols_pref if c in stations_selected.columns]
stations_selected = stations_selected[cols_exist].copy()

# --- Parseo robusto de fechas ---
for col in ['Fecha_instalacion', 'Fecha_suspension']:
    if col in stations_selected.columns:
        stations_selected[col] = pd.to_datetime(stations_selected[col], errors='coerce', dayfirst=True)

# --- Filtrar: eliminar suspendidas antes de 2010-01-01 ---
corte = pd.Timestamp('2010-01-01')
antes = len(stations_selected)
# Mantener registros sin suspensión o con suspensión >= corte
mask_keep = stations_selected['Fecha_suspension'].isna() | (stations_selected['Fecha_suspension'] >= corte)
stations_selected = stations_selected.loc[mask_keep].reset_index(drop=True)
eliminadas = antes - len(stations_selected)

print(f"Estaciones eliminadas por suspensión antes de 2010-01-01: {eliminadas}")
print(stations_selected.info())


Archivo detectado: Catalogo_Nacional_de_Estaciones_del_IDEAM_20251024.csv
Estaciones eliminadas por suspensión antes de 2010-01-01: 43
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Departamento       129 non-null    object        
 1   Municipio          129 non-null    object        
 2   Codigo             129 non-null    int64         
 3   Nombre             129 non-null    object        
 4   Categoria          129 non-null    object        
 5   Tecnologia         129 non-null    object        
 6   LONGITUD           129 non-null    float64       
 7   LATITUD            129 non-null    float64       
 8   Altitud            129 non-null    object        
 9   Fecha_instalacion  128 non-null    datetime64[ns]
 10  Fecha_suspension   12 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(

In [3]:
# --- Conteo de estaciones por Categoria ---
if 'Categoria' not in stations_selected.columns:
    raise KeyError("La columna 'Categoria' no existe en stations_selected.")

categoria_counts = (
    stations_selected['Categoria']
    .value_counts(dropna=False)
    .rename_axis('Categoria')
    .reset_index(name='Estaciones')
    .sort_values('Estaciones', ascending=False)
)
categoria_counts['Porcentaje'] = (categoria_counts['Estaciones'] / categoria_counts['Estaciones'].sum() * 100).round(2)

print(categoria_counts)


                Categoria  Estaciones  Porcentaje
0           Pluviométrica          46       35.66
1     Climática Principal          27       20.93
2            Limnimétrica          16       12.40
3     Climática Ordinaria          13       10.08
4            Limnigráfica          10        7.75
5           Pluviográfica           9        6.98
6  Meteorológica Especial           6        4.65
7       Agrometeorológica           1        0.78
8     Sinóptica Principal           1        0.78
